In [ ]:
# import xgboost  as xgb
import numpy as np
import pandas as pd

### Technical Indicators

In [3]:
import numpy as np
import pandas as pd

def RSI(series, period=14):
    """Calculate Relative Strength Index"""
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    
    avg_gain = gain.ewm(com=period, adjust=False).mean()
    avg_loss = loss.ewm(com=period, adjust=False).mean()
    
    rs = avg_gain/avg_loss
    rsi = 100 - (100 / (1+rs))
    
    return rsi

def MACD(series):
    """Calculate Moving Average Convergence Divergence"""
    ema_12 = series.ewm(span=12, adjust=False).mean()
    ema_26 = series.ewm(span=26, adjust=False).mean()
    macd = ema_12 - ema_26
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal

def ROC(series, period=14):
    """Calculate Rate of Change"""
    return (series - series.shift(period)) * 100 / series.shift(period)

def STO_OS(close, low, high, period=14):
    """Calculate Stochastic Oscillator %K"""
    low_14 = low.rolling(period).min()
    high_14 = high.rolling(period).max()
    return (close - low_14) * 100 / (high_14 - low_14)

def CCI(close, low, high, period=20):
    """Calculate Commodity Channel Index"""
    tp = (high + low + close) / 3
    sma_tp = tp.rolling(window=period).mean()
    mean_dev = tp.rolling(window=period).apply(lambda x: (abs(x - x.mean())).mean())
    
    cci = (tp - sma_tp) / (0.015 * mean_dev)
    return cci

def DIX(close, period=14, method='SMA'):
    """Calculate Distance Index"""
    if method.upper() == 'SMA':
        ma = close.rolling(window=period).mean()
    elif method.upper() == 'EMA':
        ma = close.ewm(span=period, adjust=False).mean()
    else:
        raise ValueError('Method must be SMA or EMA')
    
    dix = (close - ma) * 100 / ma
    return dix

def ATR(close, low, high, period=14):
    """Calculate Average True Range"""
    prev_close = close.shift(1)
    tr1 = high - low
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    atr = tr.ewm(span=period, adjust=False).mean()
    return atr

def OBV(close, volume):
    """Calculate On-Balance Volume"""
    if volume.sum() == 0:
        return pd.Series(0, index=close.index)
        
    price_change_sign = np.sign(close.diff().fillna(0))
    signed_volume = volume * price_change_sign
    obv = signed_volume.cumsum()
    return obv

def CMF(close, low, high, volume, period=20):
    """Calculate Chaikin Money Flow"""
    if volume.sum() == 0:
        return pd.Series(0, index=close.index)
        
    try:
        # Avoid division by zero
        range_diff = high - low
        range_diff = range_diff.replace(0, np.nan)  
        
        mfm = ((2 * close - high - low) / range_diff).fillna(0)
        mfv = mfm * volume
        cmf = mfv.rolling(window=period).sum() / volume.rolling(window=period).sum().replace(0, np.nan).fillna(1)
        return cmf.fillna(0)
    except Exception as e:
        print(f"Error calculating CMF: {e}")
        return pd.Series(0, index=close.index)

def calculate_technical_indicators(df):
    """Calculate all technical indicators for a given dataframe"""
    result = pd.DataFrame(df, index=df.index)
    
    # Basic indicators that don't need volume
    result['RSI'] = RSI(df['Close'])
    result['MACD'] = MACD(df['Close'])[0]
    result['ROC'] = ROC(df['Close'])
    result['%K'] = STO_OS(df['Close'], df['Low'], df['High'])
    result['CCI'] = CCI(df['Close'], df['Low'], df['High'])
    result['DIX'] = DIX(df['Close'])
    result['ATR'] = ATR(df['Close'], df['Low'], df['High'])
    
    # Volume-based indicators (handle possible zeros)
    result['OBV'] = OBV(df['Close'], df['Volume'])
    result['CMF'] = CMF(df['Close'], df['Low'], df['High'], df['Volume'])
    
    # Replace any NaNs or infinities with zeros
    result.replace([np.inf, -np.inf], 0, inplace=True)
    
    return result

### Read Data

In [12]:
df = pd.read_csv('../data/sp500_stock_prices_2000_2025.csv', parse_dates=['Date'], skiprows=[1]).drop(columns=['Adj Close'])
df.set_index('Date', inplace=True)

In [5]:
df

,Close,High,Low,Open,Volume
Date,,,,,
2000-01-03,1455.219971,1478.000000,1438.359985,1469.250000,931800000
2000-01-04,1399.420044,1455.219971,1397.430054,1455.219971,1009000000
2000-01-05,1402.109985,1413.270020,1377.680054,1399.420044,1085500000
2000-01-06,1403.449951,1411.900024,1392.099976,1402.109985,1092300000
2000-01-07,1441.469971,1441.469971,1400.729980,1403.449951,1225200000
...,...,...,...,...,...
2025-04-23,5375.859863,5469.689941,5356.169922,5395.919922,5371390000
2025-04-24,5484.770020,5489.399902,5371.959961,5381.379883,4697710000
2025-04-25,5525.209961,5528.109863,5455.859863,5489.729980,4236580000


In [ ]:
def create_technical_features(df):
    """
    Create technical indicators for stock price data.
    
    Parameters:
    df (DataFrame): DataFrame with stock price data, must include 'Price' column
    
    Returns:
    DataFrame: Original dataframe with added technical indicators
    """
    # Make a copy to avoid modifying the original dataframe
    df_tech = df.copy()
    
    # Ensure Price column exists
    if 'Price' not in df_tech.columns:
        raise ValueError("DataFrame must contain 'Price' column")
    
    # Calculate EMA (Exponential Moving Average)
    df_tech['EMA_9'] = df_tech['Price'].ewm(span=9, adjust=False).mean()
    
    # Calculate SMAs (Simple Moving Averages)
    df_tech['SMA_5'] = df_tech['Price'].rolling(window=5).mean()
    df_tech['SMA_15'] = df_tech['Price'].rolling(window=15).mean()
    df_tech['SMA_30'] = df_tech['Price'].rolling(window=30).mean()
    
    # Calculate RSI (Relative Strength Index)
    # First calculate price differences
    delta = df_tech['Price'].diff()
    
    # Create gain (upward movement) and loss (downward movement) series
    gain = delta.copy()
    gain[gain < 0] = 0
    loss = -delta.copy()
    loss[loss < 0] = 0
    
    # Calculate SMA of gains and losses
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    
    # Calculate RS (Relative Strength) and RSI
    rs = avg_gain / avg_loss
    df_tech['RSI'] = 100 - (100 / (1 + rs))
    
    # Calculate MACD (Moving Average Convergence Divergence)
    ema_12 = df_tech['Price'].ewm(span=12, adjust=False).mean()
    ema_26 = df_tech['Price'].ewm(span=26, adjust=False).mean()
    df_tech['MACD'] = ema_12 - ema_26
    
    # Calculate MACD Signal line
    df_tech['MACD_SIGNAL'] = df_tech['MACD'].ewm(span=9, adjust=False).mean()
    
    return df_tech

# Apply the function to create technical features
df_with_features = create_technical_features(df)

# Display the first few rows to verify
df_with_features.head()

### Process Data

In [13]:
processed_df = calculate_technical_indicators(df)

### Data Labeling

In [14]:
# Target is pct Change of the next day's close
processed_df['Target'] = processed_df['Close'].pct_change().shift(-1)

### Train Test Split

In [15]:
target_col = 'Target'
X = processed_df.drop(columns=[target_col])
y = processed_df[target_col].dropna()
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [18]:
from sklearn.ensemble import RandomForestRegressor


param_grid_rf = {
        'n_estimators': [100, 150, 200, 300],
        'max_depth': [10, 15, 20, 12],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 1.0],
        'random_state': [42, 43, 44, 45],
    }

rf = RandomForestRegressor(n_jobs=-1)


In [19]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit


time_series_cv = TimeSeriesSplit(n_splits=10, gap=0)

grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf,
                                  cv=time_series_cv,
                                  scoring='neg_mean_squared_error',
                                  verbose=2,
                                  n_jobs=-1)

In [21]:
grid_search_rf.fit(X_train, y_train)

Fitting 10 folds for each of 1152 candidates, totalling 11520 fits


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
             estimator=RandomForestRegressor(n_jobs=-1), n_jobs=-1,
             param_grid={'max_depth': [10, 15, 20, 12],
                         'max_features': ['sqrt', 1.0],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 150, 200, 300],
                         'random_state': [42, 43, 44, 45]},
             scoring='neg_mean_squared_error', verbose=2)

In [ ]:
best_params = grid_search_rf.best_params_
print(f"\nBest parameters found for RandomForest: {best_params}")
print(f"Best cross-validation score (Negative MSE with TimeSeriesSplit): {grid_search_rf.best_score_}")